In [5]:
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn.model_selection import train_test_split

In [4]:
# Create Function To Process

In [6]:
#Only columns we'll look into
cols_to_use = ['watchlist_count', 'trending_score', 'sentiment_change', 'volume_change', 'processing_time', 'volume', 'price_change', 'close']

In [8]:
def preprocess_dataframe(df, cols_to_use = cols_to_use):
    #create stock_id and time column to sort
    df['temp_sort'] = df['st_id'].astype(str) + ' ' +df['Date']
    
    #sort whole df with that 
    df = df.sort_values(by=['temp_sort'])

    #Create Price Change column
    df['price_change'] = df['close'].pct_change()
    
    #the first instance is always 0
    df['price_change'].fillna(0, inplace=True)
    
    #save index
    df_index = df['_id']
    df = df.set_index('_id')

    df1 = deepcopy(df[cols_to_use]
    return df1

In [10]:
def preprocess_minmax(df):
    '''
    we can put the different scalers
    and have user pick the name of the scaler and automatically scale
    we would instruct user to import all scalers first
    
    '''
    #save index first
    df_index = df.index
    df_cols = list(df.columns)

    #pick scaler
    scaler = MinMaxScaler()
    
    #fit scaler to dataframe
    scaler.fit(df)

    #transform and save
    df = scaler.transform(df)
    
    #turn into dataframe
    df= pd.DataFrame(df)

    df['_id'] = df_index

    df = df.set_index('_id')
    return df
    
    
    

In [13]:
def shuffle_n_split(df):
    # Shuffle
    df = df.sample(frac = 1)

    #shuffle and split
    X = df[df.columns.difference(['close'])]
    y = df['close']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    
    return X, y, X_train, X_test, y_train, y_test



In [ ]:
#Scikit Learn process
CCL = preprocess_dataframe(CCL, cols_to_use)
CCL = all_stocks_intraday_df[all_stocks_intraday_df['st_id'] == 4681]
X, y, X_train, X_test, y_train, y_test = shuffle_n_split(CCL)

X_train = preprocess_minmax(X_train)
X_test = preprocess_minmax(X_test)

regr = LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
mse = mean_squared_error(y_pred, y_test)
rmse = mse**.5
y_test.describe()

In [ ]:
#statmodels


In [ ]:
# regr = LinearRegression()
# regr.fit(X_train, y_train)
# y_pred = regr.predict(X_test)
# mse = mean_squared_error(y_pred, y_test)
# mse**.5
# y_test.describe()

In [ ]:
# statsmodel approach
train_set = X_train.merge(pd.DataFrame(y_train), left_index = True, right_index = True)
# pd.merge(X_train, X_test, join = 'inner', on= 'index')

predictors = '+'.join(train_set.columns.difference(['close']))
my_formula = 'close ~' + predictors
lin_reg_fit = sm.ols(formula = my_formula, data = train_set).fit()
lin_reg_fit.summary()